In [1]:
# Imports
import torch

# Prologue
import dlc_practical_prologue as prologue

In [3]:
# Load data
N = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)